In [10]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from methylVA.mnist.dataset import get_methyl_data_loaders

data_id = 0.05
batch_size = 128

n_features = 30579
n_samples_train = 33360
n_samples_test = 3707

train_data_path = f"../data/random_data/train_data_{n_samples_train}_{n_features}.pkl"
test_data_path = f"../data/random_data/test_data_{n_samples_test}_{n_features}.pkl"

train_metadata_path = f"../data/random_data/train_metadata_{n_samples_train}_{n_features}.pkl"
test_metadata_path = f"../data/random_data/test_metadata_{n_samples_test}_{n_features}.pkl"


train_loader, test_loader = get_methyl_data_loaders(
    train_data_path,
    train_metadata_path,
    test_data_path,
    test_metadata_path,
    batch_size=batch_size
)


In [3]:
data_batch, _ = next(iter(train_loader))


num_train_rows = len(train_loader.dataset)
num_test_rows = len(test_loader.dataset)

print("Number of features in each dataset:", data_batch.shape[1])
print("Number of rows in the training dataset:", num_train_rows)
print("Number of rows in the test dataset:", num_test_rows)

Number of features in each dataset: 30579
Number of rows in the training dataset: 33360
Number of rows in the test dataset: 3707


In [5]:
from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter

from methylVA.mnist.model import VAE
from methylVA.mnist.training import train, test

input_dim = data_batch.shape[1]
learning_rate = 1e-3
weight_decay = 1e-2
num_epochs = 100
latent_dim = 32
hidden_dim = 2048
kl_weight = 1.0
name = f'VAE_random_data_{data_id}_latent_{latent_dim}_kl_{kl_weight}'



In [6]:

writer_train = SummaryWriter(f'../experiments/{name}/train/{datetime.now().strftime("%Y%m%d-%H%M%S")}')
writer_test = SummaryWriter(f'../experiments/{name}/test/{datetime.now().strftime("%Y%m%d-%H%M%S")}')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(input_dim=input_dim, latent_dim=latent_dim, hidden_dim=hidden_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [8]:
model

VAE(
  (encoder): Sequential(
    (0): Linear(in_features=30579, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=1024, bias=True)
    (3): SiLU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): SiLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SiLU()
    (8): Linear(in_features=256, out_features=64, bias=True)
  )
  (softplus): Softplus(beta=1.0, threshold=20.0)
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=256, bias=True)
    (1): SiLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): SiLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=2048, bias=True)
    (7): SiLU()
    (8): Linear(in_features=2048, out_features=30579, bias=True)
    (9): Sigmoid()
  )
)

In [ ]:
from methylVA.mnist.training import train, test


prev_updates = 0
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    prev_updates = train(model, train_loader, optimizer, prev_updates, writer=writer_train)
    test(model, test_loader, prev_updates, writer=writer_test)

Epoch 1/100


  0%|          | 1/261 [00:00<03:21,  1.29it/s]

Step 0, (N samples: 0), Loss: 21200.0684, (Recon: 21196.5742, KLD: 3.4940), Gradient norm: 5.8527


 39%|███▉      | 102/261 [00:20<00:31,  5.07it/s]

Step 100, (N samples: 12,800), Loss: 21196.1973, (Recon: 21196.1875, KLD: 0.0088), Gradient norm: 5.2876


...

 43%|████▎     | 112/261 [00:21<00:28,  5.27it/s]

Step 23,600, (N samples: 3,020,800), Loss: 21196.1816, (Recon: 21196.1816, KLD: 0.0001), Gradient norm: 20.8328


 81%|████████  | 212/261 [00:40<00:09,  5.22it/s]

Step 23,700, (N samples: 3,033,600), Loss: 21196.2305, (Recon: 21196.2305, KLD: 0.0001), Gradient norm: 19.7289


 97%|█████████▋| 252/261 [00:48<00:01,  5.21it/s]

In [12]:
def pearson_correlation(original_x, x_hat, mask=None):
    
    # Calculate mean and standard deviation
    mean_x = torch.mean(original_x)
    mean_x_hat = torch.mean(x_hat)
    
    std_x = torch.std(original_x)
    std_x_hat = torch.std(x_hat)
    
    # Calculate covariance
    covariance = torch.mean((original_x - mean_x) * (x_hat - mean_x_hat))
    
    # Calculate Pearson correlation
    correlation = covariance / (std_x * std_x_hat)
    
    return correlation.item()


correlations = []

# Disable gradient calculation for efficiency
with torch.no_grad():
    for batch in train_loader:
        # Unpack the batch to get the input data; adjust if there are labels
        original_x, _ = batch  # Adjust this if your batch contains more items
        original_x = original_x.to(device)  # Move input to device if using GPU

        # Get the model output for the entire batch
        out = model(original_x)
        x_hat_batch = out.x_recon

        # Compute Pearson correlation for each element in the batch
        for i in range(original_x.size(0)):  # Loop over each sample in the batch
            original_sample = original_x[i]  # Select the i-th sample
            x_hat_sample = x_hat_batch[i]    # Select the corresponding output sample

            # Compute Pearson correlation for the sample
            correlation = pearson_correlation(original_sample, x_hat_sample)
            correlations.append(correlation)
            # print(f"Sample {i} Pearson Correlation:", correlation)

# Optionally, compute average correlation across all elements
average_correlation = sum(correlations) / len(correlations)
print("Average Pearson Correlation across all samples:", average_correlation)


Average Pearson Correlation across all samples: 0.003047886254953311
